# Descripción del data set Financial data of 4400+ public companies

✅ ¿Qué es?

Se trata de un conjunto de datos que contiene información financiera de más de 4 400 empresas públicas.  

Los datos fueron raspados (“scraped”) de Yahoo Finance, lo que incluye los estados financieros — el balance general (“balance sheet”), el estado de resultados (“income statement”), y el flujo de caja (“cash flow statement”).  

Incluye datos tanto anuales como trimestrales para un período reciente (aproximadamente los últimos 4 años) para cada empresa.  


# ¿Qué contiene exactamente?

Algunas de las características clave de la base de datos:

- Cada empresa viene identificada con su ticker o símbolo bursátil (o equivalente) y posiblemente con su nombre, industria, etc.
- Para cada empresa, los datos incluyen:
    - Estado de resultados: ingresos, beneficios, gastos, etc.
    - Balance general: activos, pasivos, capital contable, etc.
    - Flujo de caja: flujos operativos, de inversión, de financiación, etc.
- Los datos están organizados para que puedas ver la evolución por trimestre y por año, lo que permite análisis de tendencias.
- Se pueden usar variables financieras como predictors (por ejemplo: activos, pasivos, ingresos) y variables objetivo como beneficio, rentabilidad, crecimiento, etc.

# Paso 1 — Reconocer el dataset

1. Objetivo: investigar el dataset entregado en la asignación 2, revisar el nombre de cada columna y documentar qué representa cada una (tipo, unidad, periodicidad, observaciones).

2. Pasos recomendados:
    - Cargar el archivo entregado (por ejemplo: df = pd.read_csv(...)) y listar columnas: df.columns
    - Para cada columna: buscar su significado (glosario, documentación de la fuente, o inspección de valores).
    - Anotar tipo de dato, unidad (USD, porcentaje, entero), periodicidad (trimestral/anual) y cualquier observación (por ejemplo: calculada, acumulada, neta/bruta).
    - Generar la lista final con nombre y descripción clara y breve.

3. Ejemplo de Plantilla para la lista (reemplazar con las columnas reales del dataset):

- ticker: Identificador bursátil de la empresa (string). Ejemplo: "AAPL".
- fiscal_date: Fecha del periodo financiero (YYYY-MM-DD). Indica el cierre del trimestre/año.
- revenue: Ingresos netos durante el periodo (num, USD). Periodicidad: trimestral/anual.
- gross_profit: Beneficio bruto (num, USD). Definición: ingresos menos costo de ventas.
- operating_income: Resultado operativo (num, USD). Incluye gastos operativos.
- net_income: Beneficio neto después de impuestos (num, USD).


4. Resultado esperado:
    - Un listado documentado con cada columna del dataset y su descripción (puede entregarse como tabla o como lista de pares nombre→descripción).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Paso 1: Cargar y explorar el dataset
file_path = 'balanceSheetHistory_annually.csv'
raw_df = pd.read_csv(file_path)

print(f"Filas: {raw_df.shape[0]}, Columnas: {raw_df.shape[1]}")
print("Columnas disponibles:")
print(raw_df.columns.tolist())

# Vista rápida de los primeros registros
raw_df.head()

# Paso 2 — Seleccionar columnas relevantes

Después de haber cargado el dataset, elimina todas las columnas y quédate solo con: `stock`, `endDate` y `cash`.

- Verifica que los nombres y la capitalización de las columnas sean correctos antes de seleccionar.


In [ ]:
# Paso 2: Nos quedamos solo con stock, endDate y cash
filtered_df = raw_df[['stock', 'endDate', 'cash']].copy()

# Eliminamos filas sin valor de cash y convertimos la fecha a datetime
filtered_df = filtered_df.dropna(subset=['cash'])
filtered_df['endDate'] = pd.to_datetime(filtered_df['endDate'])

# Ordenar por empresa y fecha para mantener coherencia temporal
filtered_df = filtered_df.sort_values(['stock', 'endDate']).reset_index(drop=True)

print(filtered_df.head())
print("Empresas distintas con datos de cash:", filtered_df['stock'].nunique())

# Paso 3 — Separar por empresas y elegir 3 para predecir su cantidad de dinero por fecha

- Objetivo: crear series temporales por empresa usando las columnas `stock`, `endDate` y `cash` y seleccionar 3 empresas para modelar y predecir `cash` por fecha.
- Pasos recomendados:
    - Verificar que las columnas `stock`, `endDate` y `cash` existen y están limpias (sin valores nulos o con imputación cuando sea necesario).
    - Ordenar el dataframe por `stock` y `endDate` (ascendente) para obtener la serie temporal de cada empresa.
    - Agrupar por `stock` y crear un subset por empresa.
    - Elegir 3 empresas con suficientes observaciones (p. ej. mayor número de fechas disponibles o relevancia del negocio).
    - Para cada empresa seleccionada, preparar los datos de entrenamiento/validación (features temporales, ventanas, lag, etc.) y definir la variable objetivo `cash` por `endDate`.

In [ ]:
# Paso 3: Crear series por empresa y elegir 3 con más observaciones
stock_counts = filtered_df['stock'].value_counts()
selected_stocks = stock_counts.head(3).index.tolist()

series_por_empresa = {stock: filtered_df[filtered_df['stock'] == stock].copy() for stock in selected_stocks}

print("Empresas seleccionadas (top 3 por cantidad de registros):", selected_stocks)
for stock, data in series_por_empresa.items():
    print(f"Resumen para {stock}: {len(data)} registros, rango de fechas {data['endDate'].min().date()} a {data['endDate'].max().date()}")

# Paso 4 — Grafique tiempo vs dinero de las 3 empresas en 3 gráficas diferentes

Objetivo: visualizar la serie temporal de `cash` frente a `endDate` para cada una de las 3 empresas seleccionadas, colocando cada empresa en una gráfica independiente.

Pasos recomendados:
- Asegurarse de tener el DataFrame con las columnas `stock`, `endDate` y `cash` y ordenado por `stock` y `endDate` ascendente.
- Convertir `endDate` a tipo fecha: `df['endDate'] = pd.to_datetime(df['endDate'])`.
- Seleccionar las 3 empresas elegidas: p. ej. `stocks = ['AAA','BBB','CCC']` y crear un subset por cada `stock`.
- Para cada empresa, graficar `endDate` en el eje x y `cash` en el eje y en una figura separada.
- Configurar títulos, etiquetas de ejes y formato de fechas (rotar etiquetas si es necesario). Añadir grid y leyenda si procede.
- Opcional: usar subplots (3 filas x 1 columna) para mostrar las 3 gráficas en la misma figura o generar 3 figuras individuales según preferencia.
- Guardar las figuras si es necesario: `plt.savefig('cash_stock_AAA.png', bbox_inches='tight')`.

Ejemplo de librerías a usar: matplotlib, seaborn o plotly para interactividad.

In [ ]:
# Paso 4: Gráficas tiempo vs cash para las 3 empresas seleccionadas
fig, axes = plt.subplots(3, 1, figsize=(10, 14), sharex=True)

for ax, stock in zip(axes, selected_stocks):
    data = series_por_empresa[stock]
    ax.plot(data['endDate'], data['cash'], marker='o')
    ax.set_title(f"{stock}: evolución anual de cash")
    ax.set_ylabel('Cash')
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Fecha')
plt.tight_layout()
plt.show()

# Paso 5 — División 80/20 (entrenamiento / prueba)

Objetivo: separar los datos en 80% para entrenamiento y 20% para prueba respetando la estructura temporal por empresa (sin hacer shuffle).

Recomendaciones:
- Asegúrate de tener las columnas `stock`, `endDate` y `cash` y que el dataframe esté ordenado por `stock` y `endDate` (ascendente).
- Para series temporales por empresa, usa una división basada en tiempo: los primeros 80% de observaciones de cada `stock` → train; los últimos 20% → test.



In [ ]:
# Paso 5: División 80/20 respetando el orden temporal por empresa
from math import ceil

splits = {}
for stock, data in series_por_empresa.items():
    data = data.sort_values('endDate').copy()
    # Convertimos fecha a número ordinal para poder usarla como feature temporal
    data['time_index'] = data['endDate'].map(pd.Timestamp.toordinal)

    cutoff = int(len(data) * 0.8)
    train = data.iloc[:cutoff]
    test = data.iloc[cutoff:]

    splits[stock] = {
        'train': train,
        'test': test
    }
    print(f"{stock}: train {train.shape}, test {test.shape}")

# Paso 6 — crea y entrena el modelo de regresión lineal para predecir `cash` por empresa

Objetivo: entrenar un modelo de regresión lineal por cada `stock` usando la serie temporal (respetando orden temporal) y evaluar en el 20% final.

Pasos recomendados:
- Preprocesamiento
    - Definir variables para el entrenamiento: y = `cash` (variable objetivo). `data` → X (conjunto de características a usar para predecir `cash`, p. ej. rezagos de `cash`, indicadores temporales, variables exógenas). Usar X e y en el entrenamiento: `model.fit(X_train, y_train)`.
    - Asegurar `endDate` como datetime y ordenar por `stock`, `endDate`.  - Asegurar `endDate` como datetime y ordenar por `stock`, `endDate`.



In [ ]:
# Paso 6: Entrenar regresión lineal para cada empresa
from sklearn.linear_model import LinearRegression

models = {}
predicciones = {}

for stock, parts in splits.items():
    X_train = parts['train'][['time_index']]
    y_train = parts['train']['cash']

    model = LinearRegression()
    model.fit(X_train, y_train)

    X_test = parts['test'][['time_index']]
    y_test = parts['test']['cash']
    y_pred = model.predict(X_test)

    models[stock] = model
    predicciones[stock] = {
        'fechas_test': parts['test']['endDate'],
        'y_test': y_test,
        'y_pred': y_pred
    }

    print(f"Modelo para {stock}: pendiente {model.coef_[0]:.4f}, intercepto {model.intercept_:.2f}")

# Paso 7 — Verifica tu modelo de regresión lineal: grafica real vs predicho para las 3 empresas

- Objetivo: comparar visualmente los valores reales de `cash` del conjunto de prueba con los valores predichos por el modelo para cada una de las 3 empresas seleccionadas.
- Requisitos: tener `endDate` como datetime, el conjunto test por cada `stock`, y las predicciones (`y_pred`) para cada test.
- Pasos recomendados:
    - Para cada empresa (stock):
        - Extraer test: filas finales (20%) ordenadas por `endDate`.
        - Obtener predicciones usando el modelo entrenado: `y_pred = model.predict(X_test)`.
        - Crear una gráfica con `endDate` en el eje x y ambos: `cash` real (línea/points) y `cash` predicho (línea punteada) en el eje y.
        - Añadir título con el ticker, leyenda, etiquetas de ejes y grid. Formatear fechas y rotar etiquetas si hace falta.
    - Opcional: mostrar las 3 series en subplots (3 filas x 1 columna) para facilitar comparación.
    - Calcular y mostrar métricas de error por empresa (MAE, RMSE, R2) bajo cada gráfico o en una tabla resumen.
- Resultado esperado: tres gráficas (una por empresa) mostrando real vs predicho y una tabla o texto con las métricas de evaluación.

In [ ]:
# Paso 7: Graficar valores reales vs predichos para el conjunto de prueba
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

for ax, stock in zip(axes, predicciones.keys()):
    fechas = predicciones[stock]['fechas_test']
    y_test = predicciones[stock]['y_test']
    y_pred = predicciones[stock]['y_pred']

    ax.plot(fechas, y_test, marker='o', label='Real')
    ax.plot(fechas, y_pred, marker='x', linestyle='--', label='Predicción')
    ax.set_title(f"{stock} - Cash en test")
    ax.set_xlabel('Fecha')
    ax.set_ylabel('Cash')
    ax.legend()
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Paso 8 — Verificación de modelos (MSE, RMSE, R2)

Instrucciones breves:
- Asegúrese de tener para cada empresa: y_test (valores reales) y y_pred (predicciones).
- Calcular métricas con sklearn: mean_squared_error(y_test, y_pred), RMSE = sqrt(MSE), r2_score(y_test, y_pred).
- Presentar los resultados en una tabla por empresa y añadir una conclusión corta.

Empresa 1 (ticker: AAA)
| Métrica | Valor |
|---|---|
| MSE |  |
| RMSE |  |
| R2 |  |

Empresa 2 (ticker: BBB)
| Métrica | Valor |
|---|---|
| MSE |  |
| RMSE |  |
| R2 |  |

Empresa 3 (ticker: CCC)
| Métrica | Valor |
|---|---|
| MSE |  |
| RMSE |  |
| R2 |  |

Pequeña conclusión:
- Comparar RMSE/MSE absolutos para evaluar error en unidades de `cash`; RMSE más bajo = mejor precisión.
- R2 indica proporción de varianza explicada (cercano a 1 → buen ajuste; cercano a 0 o negativo → mal ajuste).
- Si alguno de los modelos muestra RMSE alto o R2 bajo, considerar: más features (rezagos, variables temporales), regularización, transformación de la serie o modelos no lineales.

In [ ]:
# Paso 8: Métricas de verificación (MSE, RMSE, R2)
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

metricas = []
for stock, valores in predicciones.items():
    y_test = valores['y_test']
    y_pred = valores['y_pred']

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    metricas.append({
        'stock': stock,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2
    })

metricas_df = pd.DataFrame(metricas).set_index('stock')
print(metricas_df.round(2))

print("Conclusión rápida: valores de R2 cercanos a 1 indican buen ajuste. Si son bajos, el modelo lineal simple no captura bien la tendencia temporal de cash para esa empresa.")

## Preguntas de Analisis 

1. **Variables utilizadas:** solo `endDate` transformada a índice temporal como feature y `cash` como variable objetivo. Se eligió el tiempo porque buscamos proyectar la tendencia temporal del efectivo por empresa.
2. **División temporal 80/20:** se ordenaron los datos por `endDate` y se separó el 80% inicial como entrenamiento y el 20% final como prueba por cada empresa, evitando `shuffle` para no romper la secuencia temporal.
3. **MSE vs RMSE:** el MSE promedia el cuadrado del error, penalizando fuertemente desviaciones grandes; el RMSE es la raíz del MSE y vuelve la métrica a las mismas unidades de `cash`, facilitando la interpretación del error promedio.
4. **Interpretación de R²:** indica la proporción de la variabilidad de `cash` explicada por el modelo. Valores cercanos a 1 muestran buen ajuste; valores cercanos a 0 o negativos indican que la tendencia lineal en el tiempo no explica bien los cambios de efectivo.